In [25]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from time import sleep
from bs4 import BeautifulSoup
import requests
import re

### Utiliza las esperas explícitas de Selenium para esperar hasta que el elemento esté visible y se pueda interactuar con él.(Opcional)

In [26]:
from selenium.webdriver.common.by import By   
from selenium.webdriver.support.ui import WebDriverWait  
from selenium.webdriver.support import expected_conditions as EC

-----------------------------------Lo de arriba es opcional--------------------------------------------------------

In [27]:
pd.set_option('display.max_columns', None)

#### Utiliza esperas explícitas. Este método es preferido sobre las esperas implícitas o sleep(), ya que hace que tu script espere de manera más inteligente a que los elementos estén listos para la interacción, reduciendo así la probabilidad de errores y haciendo que tu código sea más eficiente y confiable. (Opcional)

In [28]:
# driver.maximize_window()
# sleep(2)

# # Esperar hasta que el botón sea clickeable
# WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#wt-cli-accept-all-btn"))).click()

# sleep(5)

# # Esperar hasta que el enlace BLOG sea clickeable
# WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.LINK_TEXT, 'BLOG'))).click()

# sleep(5)


----------------------Lo se arriba es opcional-----------------------

In [29]:
# Inicializar un navegador web Chrome a través de WebDriver
driver = webdriver.Chrome()

# Navegar a la página web de DIA
url = "https://www.dia.es/"
driver.get(url)

# Maximizar la ventana del navegador para mostrar la página completa
driver.maximize_window()

# Esperar 5 segundos para que la página cargue completamente
sleep(2)

# Aceptar las cookies haciendo clic en el botón correspondiente
driver.find_element("css selector", "#onetrust-accept-btn-handler").click()

# Esperar 5 pero entra si lo consigue hacer antes.
sleep(2)

# Entrar en el apartado de productos
driver.find_element("css selector", "#app > div > div > div > div.home-view__header > div.dia-header > div.dia-header__section.dia-header__section--start > div > button").click()


sleep(5)

# Entrar en la categoría "carnicería
driver.find_element("xpath", "//span[@class='category-item__title' and contains(text(), 'Carnicería')]").click()
#Este XPath seleccionará un <span> con la clase category-item__title que también contiene el texto "Carnicería". De esta manera, estás asegurando que el clic se realice en el elemento que específicamente se refiere a "Carnicería", y no en otro elemento que tenga la misma clase pero diferente contenido de texto.


# Crear un diccionario para almacenar información de productos
diccionario_dia = {
    "nombre_producto": [],
    "precio_kilo": [],
    "precio": [],
    "kilocalorias": [],
    "ingredientes": [],
    "imagen": []
}

# Iterar a través de productos del 1 al 7
for i in range(1,5):
    

    # Hacer clic en el producto para ver detalles
    driver.find_element("css selector", f"#app > div > div > div.categories-layout__content > div.categories-layout__right-content > div.plp-view > div:nth-child(3) > div > div > ul > li:nth-child({i}) > div > div.search-product-card__top-section > div.search-product-card__top-section-content > a.search-product-card__product-link").click()
    
    # Esperar 2 segundos después de aceptar las cookies
    sleep(2)
    
    # Extraer información del producto y agregarla al diccionario
    diccionario_dia["nombre_producto"].append(driver.find_element("css selector", "#app > div > div > div > div.pdp-view__content > div.pdp-view__left-content > div.pdp-view__info > div.product-summary > h2").text)
    diccionario_dia["precio"].append(driver.find_element("css selector", "#app > div > div > div > div.pdp-view__content > div.pdp-view__right-content > div > div.buy-box.primary-information__buy-box > div.buy-box__prices-container > div.buy-box__prices > p.buy-box__active-price").text)
    diccionario_dia["precio_kilo"].append(driver.find_element("css selector", "#app > div > div > div > div.pdp-view__content > div.pdp-view__right-content > div > div.buy-box.primary-information__buy-box > div.buy-box__prices-container > div.buy-box__prices > p.buy-box__price-per-unit").text)
    diccionario_dia["kilocalorias"].append(driver.find_element("css selector", "#app > div > div > div > div.pdp-view__content > div.pdp-view__left-content > div.pdp-view__info > div.nutritional > ul > li > div > div.nutritional-values__title-energy > div > p:nth-child(3)").text)
    diccionario_dia["ingredientes"].append(driver.find_element("css selector", "#app > div > div > div > div.pdp-view__content > div.pdp-view__left-content > div.pdp-view__info > div.text-section > h2").text)
    diccionario_dia["imagen"].append(driver.find_element("css selector", "#app > div > div > div > div.pdp-view__content > div.pdp-view__left-content > div.product-images > div > div > div > img.product-image.product-images__second-image").get_attribute("src"))
    
    # Esperar 2 segundos antes de volver a la lista de productos
    sleep(2)
    driver.back()

# Cerrar el navegador
driver.close()

In [30]:
diccionario_dia["nombre_producto"]

['Pechuga de pavo cocida',
 'Jamoncitos de pollo.',
 'Longaniza de pollo',
 'Pechuga de pollo entera.']

In [31]:
# utilizando el método "pd.DataFrame()" que aprendiemos en las lecciones pasadas, convertimos el diccionario a un DataFrame para poder seguir trabajando con el 
df_dia = pd.DataFrame(diccionario_dia)

# mostrar los resultados de la extracción de datos almacenados en un DataFrame
df_dia

,nombre_producto,precio_kilo,precio,kilocalorias,ingredientes,imagen
0,Pechuga de pavo cocida,"(8,43 €/KILO)","3,37 €",74kcal,INGREDIENTES,https://www.dia.es/product_images/273810/27381...
1,Jamoncitos de pollo.,"(3,35 €/KILO)","2,68 €",155kcal,INGREDIENTES,https://www.dia.es/product_images/4211/4211_FR...
2,Longaniza de pollo,"(7,48 €/KILO)","2,99 €",155kcal,INGREDIENTES,https://www.dia.es/product_images/275700/27570...
3,Pechuga de pollo entera.,"(6,99 €/KILO)","6,99 €",114kcal,INGREDIENTES,https://www.dia.es/product_images/261371/26137...
